# Network Hardening con Automated Planning

Sistema di pianificazione automatica per la messa in sicurezza di infrastrutture di rete.

**Obiettivo:** Chiudere tutte le porte di rete considerate insicure, minimizzando il costo operativo e garantendo la continuità dei servizi critici.

---

## Indice

1. [Setup e Installazione](#1-setup)
2. [Configurazione del Dominio](#2-configurazione)
3. [Modello di Planning](#3-modello)
4. [Generazione Scenari](#4-scenari)
5. [Esecuzione e Risultati](#5-esecuzione)
6. [Analisi dei Risultati](#6-analisi)

---
## 1. Setup e Installazione <a id="1-setup"></a>

Installazione delle librerie necessarie per il planning automatico.

In [ ]:
!pip install -q unified-planning up-fast-downward pandas matplotlib

In [ ]:
import json
import os
import time
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from unified_planning.shortcuts import *
from unified_planning.engines import PlanGenerationResultStatus

warnings.filterwarnings('ignore')
os.makedirs('scenarios', exist_ok=True)
os.makedirs('results', exist_ok=True)

print("Librerie caricate correttamente.")

---
## 2. Configurazione del Dominio <a id="2-configurazione"></a>

### 2.1 Mapping Servizi e Porte

Ogni servizio di rete utilizza una porta specifica. Questo mapping definisce le associazioni standard.

In [ ]:
SERVICE_PORT_MAPPING = {
    'http': [80], 'https': [443], 'ssh': [22],
    'mysql': [3306], 'postgresql': [5432], 'redis': [6379], 'mongodb': [27017],
    'ftp': [21], 'telnet': [23], 'smtp': [25], 'pop3': [110], 'imap': [143],
    'tomcat': [8080], 'nodejs': [3000], 'rabbitmq': [5672],
    'prometheus': [9090], 'grafana': [3000], 'elasticsearch': [9200], 'kibana': [5601],
    'smb': [445], 'netbios': [139], 'rdp': [3389], 'vnc': [5900],
    'nfs': [2049], 'ldap': [389], 'rsync': [873], 'snmp': [161], 'tftp': [69]
}

### 2.2 Porte Alternative per la Migrazione

Alcune porte insicure possono essere migrate su alternative più sicure. Le porte senza alternativa (es. NetBIOS, SMTP, SNMP, TFTP) richiedono la disattivazione del servizio.

**Scelta progettuale:** Non tutte le porte hanno un'alternativa. Questo forza il planner a utilizzare strategie diverse in base al contesto.

In [ ]:
ALTERNATIVE_PORTS = {
    80: 8080,      # HTTP
    21: 2121,      # FTP
    23: 2323,      # Telnet
    110: 1110,     # POP3
    143: 1143,     # IMAP
    3389: 33389,   # RDP
    5900: 59000,   # VNC
    445: 4455,     # SMB
    389: 3890,     # LDAP
    2049: 20490,   # NFS
    # Porte SENZA alternativa (forzano disattivazione):
    # 25 (SMTP), 139 (NetBIOS), 161 (SNMP), 69 (TFTP)
}

ACTION_COSTS = {
    'chiudi_porta': 1,       # Costo minimo: nessun impatto sui servizi
    'disattiva_servizio': 5, # Costo alto: causa downtime
    'migra_servizio': 3      # Costo medio: richiede riconfigurazione
}

---
## 3. Modello di Planning <a id="3-modello"></a>

### 3.1 Definizione del Dominio

Il dominio utilizza 5 fluent per rappresentare lo stato della rete:

| Fluent | Tipo | Descrizione |
|--------|------|-------------|
| `porta_aperta(host, port)` | Bool | Indica se una porta è aperta |
| `servizio_attivo(host, service)` | Bool | Indica se un servizio è in esecuzione |
| `servizio_critico(host, service)` | Bool | Servizi che non possono essere fermati |
| `servizio_usa_porta(host, service, port)` | Bool | Relazione servizio-porta |
| `dipende_da(host, srv_dip, srv_base)` | Bool | Dipendenze tra servizi |

In [ ]:
class NetworkHardeningDomain:
    """Definizione del dominio di planning per network hardening."""
    
    def __init__(self):
        self.problem = Problem('network_hardening')
        
        # Tipi
        Host = UserType('Host')
        Port = UserType('Port')
        Service = UserType('Service')
        
        # Fluent
        self.fluents = {
            'porta_aperta': Fluent('porta_aperta', BoolType(), host=Host, port=Port),
            'servizio_attivo': Fluent('servizio_attivo', BoolType(), host=Host, service=Service),
            'servizio_critico': Fluent('servizio_critico', BoolType(), host=Host, service=Service),
            'servizio_usa_porta': Fluent('servizio_usa_porta', BoolType(), host=Host, service=Service, port=Port),
            'dipende_da': Fluent('dipende_da', BoolType(), host=Host, service_dipendente=Service, service_base=Service)
        }
        
        for f in self.fluents.values():
            self.problem.add_fluent(f, default_initial_value=False)
        
        self.types = {'Host': Host, 'Port': Port, 'Service': Service}

### 3.2 Azioni Disponibili

Il sistema utilizza 3 azioni:

| Azione | Costo | Precondizioni | Effetti |
|--------|-------|---------------|--------|
| `chiudi_porta` | 1 | Porta aperta, nessun servizio attivo | Porta chiusa |
| `disattiva_servizio` | 5 | Servizio attivo, non critico, nessun dipendente attivo | Servizio inattivo |
| `migra_servizio` | 3 | Servizio attivo, porta alternativa chiusa | Servizio su nuova porta |

In [ ]:
def get_services_on_port(host_data, port, mapping):
    """Restituisce i servizi che usano una specifica porta su un host."""
    return [s for s in host_data.get('services', []) if port in mapping.get(s, [])]

def get_dependent_services(host_id, service, dependencies):
    """Restituisce i servizi che dipendono dal servizio specificato."""
    host_deps = dependencies.get(host_id, {})
    return [s for s, deps in host_deps.items() if service in deps]

### 3.3 Setup del Problema

Funzione principale che costruisce il problema di planning a partire da uno scenario JSON.

In [ ]:
def setup_problem(scenario):
    """Configura il problema di planning a partire dallo scenario."""
    
    domain = NetworkHardeningDomain()
    problem = domain.problem
    objects = {}
    
    forbidden = scenario['policy']['forbidden_ports']
    critical = scenario['policy'].get('critical_services', {})
    mapping = scenario.get('service_port_mapping', SERVICE_PORT_MAPPING)
    dependencies = scenario.get('service_dependencies', {})
    
    # Creazione oggetti Host
    for h in scenario['hosts']:
        objects[h['id']] = Object(h['id'], domain.types['Host'])
        problem.add_object(objects[h['id']])
    
    # Creazione oggetti Port (incluse alternative)
    all_ports = set()
    for h in scenario['hosts']:
        all_ports.update(h['open_ports'])
    for p in forbidden:
        if p in ALTERNATIVE_PORTS:
            all_ports.add(ALTERNATIVE_PORTS[p])
    
    for p in sorted(all_ports):
        key = f'port_{p}'
        objects[key] = Object(key, domain.types['Port'])
        problem.add_object(objects[key])
    
    # Creazione oggetti Service
    all_services = set()
    for h in scenario['hosts']:
        all_services.update(h.get('services', []))
    
    for s in sorted(all_services):
        key = f'srv_{s}'
        objects[key] = Object(key, domain.types['Service'])
        problem.add_object(objects[key])
    
    # Stato iniziale
    for h in scenario['hosts']:
        host_obj = objects[h['id']]
        
        # Porte aperte
        for p in h['open_ports']:
            if f'port_{p}' in objects:
                problem.set_initial_value(domain.fluents['porta_aperta'](host_obj, objects[f'port_{p}']), True)
        
        # Servizi attivi e relazioni
        for s in h.get('services', []):
            srv_key = f'srv_{s}'
            if srv_key not in objects:
                continue
            srv_obj = objects[srv_key]
            problem.set_initial_value(domain.fluents['servizio_attivo'](host_obj, srv_obj), True)
            
            for p in mapping.get(s, []):
                if p in h['open_ports'] and f'port_{p}' in objects:
                    problem.set_initial_value(domain.fluents['servizio_usa_porta'](host_obj, srv_obj, objects[f'port_{p}']), True)
    
    # Servizi critici
    for host_id, services in critical.items():
        if host_id in objects:
            for s in services:
                if f'srv_{s}' in objects:
                    problem.set_initial_value(domain.fluents['servizio_critico'](objects[host_id], objects[f'srv_{s}']), True)
    
    # Dipendenze
    for host_id, deps_map in dependencies.items():
        if host_id not in objects:
            continue
        for s, deps in deps_map.items():
            if f'srv_{s}' not in objects:
                continue
            for base in deps:
                if f'srv_{base}' in objects:
                    problem.set_initial_value(
                        domain.fluents['dipende_da'](objects[host_id], objects[f'srv_{s}'], objects[f'srv_{base}']), True)
    
    # Generazione azioni grounded
    for h in scenario['hosts']:
        host_obj = objects[h['id']]
        
        # AZIONE: disattiva_servizio
        for s in h.get('services', []):
            srv_key = f'srv_{s}'
            if srv_key not in objects:
                continue
            srv_obj = objects[srv_key]
            dependents = get_dependent_services(h['id'], s, dependencies)
            
            action = InstantaneousAction(f"disattiva_servizio_{h['id']}_{s}")
            action.add_precondition(domain.fluents['servizio_attivo'](host_obj, srv_obj))
            action.add_precondition(Not(domain.fluents['servizio_critico'](host_obj, srv_obj)))
            
            for dep in dependents:
                if f'srv_{dep}' in objects:
                    action.add_precondition(Not(domain.fluents['servizio_attivo'](host_obj, objects[f'srv_{dep}'])))
            
            action.add_effect(domain.fluents['servizio_attivo'](host_obj, srv_obj), False)
            problem.add_action(action)
        
        # AZIONE: chiudi_porta
        for p in forbidden:
            if p not in h['open_ports']:
                continue
            port_obj = objects[f'port_{p}']
            services_on_port = get_services_on_port(h, p, mapping)
            
            action = InstantaneousAction(f"chiudi_porta_{h['id']}_p{p}")
            action.add_precondition(domain.fluents['porta_aperta'](host_obj, port_obj))
            
            for s in services_on_port:
                if f'srv_{s}' in objects:
                    action.add_precondition(Not(domain.fluents['servizio_attivo'](host_obj, objects[f'srv_{s}'])))
            
            action.add_effect(domain.fluents['porta_aperta'](host_obj, port_obj), False)
            for s in services_on_port:
                if f'srv_{s}' in objects:
                    action.add_effect(domain.fluents['servizio_usa_porta'](host_obj, objects[f'srv_{s}'], port_obj), False)
            
            problem.add_action(action)
        
        # AZIONE: migra_servizio
        for p in forbidden:
            if p not in h['open_ports'] or p not in ALTERNATIVE_PORTS:
                continue
            new_p = ALTERNATIVE_PORTS[p]
            if new_p in forbidden or f'port_{new_p}' not in objects:
                continue
            
            port_obj = objects[f'port_{p}']
            new_port_obj = objects[f'port_{new_p}']
            
            for s in get_services_on_port(h, p, mapping):
                if f'srv_{s}' not in objects:
                    continue
                srv_obj = objects[f'srv_{s}']
                
                action = InstantaneousAction(f"migra_servizio_{h['id']}_{s}_p{p}to{new_p}")
                action.add_precondition(domain.fluents['servizio_attivo'](host_obj, srv_obj))
                action.add_precondition(domain.fluents['porta_aperta'](host_obj, port_obj))
                action.add_precondition(Not(domain.fluents['porta_aperta'](host_obj, new_port_obj)))
                action.add_precondition(domain.fluents['servizio_usa_porta'](host_obj, srv_obj, port_obj))
                
                action.add_effect(domain.fluents['porta_aperta'](host_obj, port_obj), False)
                action.add_effect(domain.fluents['porta_aperta'](host_obj, new_port_obj), True)
                action.add_effect(domain.fluents['servizio_usa_porta'](host_obj, srv_obj, port_obj), False)
                action.add_effect(domain.fluents['servizio_usa_porta'](host_obj, srv_obj, new_port_obj), True)
                
                problem.add_action(action)
    
    # Goal: chiudere tutte le porte vietate
    for h in scenario['hosts']:
        for p in forbidden:
            if p in h['open_ports'] and f'port_{p}' in objects:
                problem.add_goal(Not(domain.fluents['porta_aperta'](objects[h['id']], objects[f'port_{p}'])))
    
    # Metrica di costo
    cost_map = {}
    for a in problem.actions:
        name = a.name.lower()
        if 'chiudi_porta' in name:
            cost_map[a] = ACTION_COSTS['chiudi_porta']
        elif 'disattiva_servizio' in name:
            cost_map[a] = ACTION_COSTS['disattiva_servizio']
        elif 'migra_servizio' in name:
            cost_map[a] = ACTION_COSTS['migra_servizio']
    
    if cost_map:
        problem.add_quality_metric(MinimizeActionCosts(cost_map))
    
    return problem, domain, objects

---
## 4. Generazione Scenari <a id="4-scenari"></a>

Gli scenari sono progettati per testare tutte le azioni del planner:
- **migra_servizio**: porte con alternativa disponibile
- **disattiva_servizio**: porte senza alternativa (NetBIOS, SMTP, SNMP, TFTP)
- **chiudi_porta**: porte aperte senza servizi attivi

In [ ]:
SCENARIOS = [
    {
        'name': '01_all_actions_basic',
        'description': 'Scenario base con tutte le azioni',
        'hosts': [
            ('web1', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('fileserver', [139, 445, 22], ['netbios', 'smb', 'ssh'], False),
            ('legacy', [23, 22], ['ssh'], False),
        ],
        'forbidden': [80, 139, 23],
        'critical': {},
        'dependencies': {},
    },
    {
        'name': '02_deps_all_actions',
        'description': 'Dipendenze tra servizi',
        'hosts': [
            ('frontend', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('backend', [8080, 22], ['tomcat', 'ssh'], False),
            ('mailserver', [25, 22], ['smtp', 'ssh'], False),
            ('database', [3306, 22, 23], ['mysql', 'ssh'], True),
        ],
        'forbidden': [80, 25, 23],
        'critical': {'database': ['mysql']},
        'dependencies': {
            'frontend': {'http': ['tomcat']},
            'backend': {'tomcat': ['mysql']}
        },
    },
    {
        'name': '03_enterprise_mixed',
        'description': 'Infrastruttura enterprise',
        'hosts': [
            ('web1', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('web2', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('app1', [8080, 21, 161, 22], ['tomcat', 'ftp', 'snmp', 'ssh'], False),
            ('app2', [8080, 21, 22], ['tomcat', 'ftp', 'ssh'], False),
            ('db_master', [3306, 22, 23], ['mysql', 'ssh'], True),
            ('db_slave', [3306, 22, 139], ['mysql', 'ssh'], True),
            ('storage', [445, 139, 22], ['smb', 'netbios', 'ssh'], False),
            ('monitoring', [9090, 22, 23], ['prometheus', 'ssh'], False),
        ],
        'forbidden': [80, 21, 23, 139, 161],
        'critical': {'db_master': ['mysql'], 'db_slave': ['mysql']},
        'dependencies': {
            'web1': {'http': ['tomcat']},
            'web2': {'http': ['tomcat']},
            'app1': {'tomcat': ['mysql']},
            'app2': {'tomcat': ['mysql']},
        },
    },
    {
        'name': '04_security_incident',
        'description': 'Post-breach hardening',
        'hosts': [
            ('dmz_web', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('dmz_mail', [25, 110, 143, 22], ['smtp', 'pop3', 'imap', 'ssh'], False),
            ('internal_app', [8080, 3389, 22], ['tomcat', 'rdp', 'ssh'], False),
            ('admin_jump', [5900, 69, 22], ['vnc', 'tftp', 'ssh'], False),
            ('legacy_server', [23, 139, 22], ['ssh'], False),
            ('db_primary', [3306, 22], ['mysql', 'ssh'], True),
        ],
        'forbidden': [80, 25, 23, 139, 3389, 5900, 69],
        'critical': {'db_primary': ['mysql']},
        'dependencies': {
            'dmz_web': {'http': ['tomcat']},
            'internal_app': {'tomcat': ['mysql']},
        },
    },
    {
        'name': '05_healthcare_gdpr',
        'description': 'Compliance GDPR/HIPAA',
        'hosts': [
            ('patient_portal', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('ehr_app', [8080, 443, 22], ['tomcat', 'https', 'ssh'], True),
            ('legacy_gateway', [25, 23, 22], ['smtp', 'ssh'], False),
            ('patient_db', [5432, 22], ['postgresql', 'ssh'], True),
            ('medical_files', [445, 139, 22], ['smb', 'netbios', 'ssh'], False),
            ('admin_ws', [3389, 161, 22], ['rdp', 'snmp', 'ssh'], False),
            ('audit_db', [3306, 22, 139], ['mysql', 'ssh'], True),
        ],
        'forbidden': [80, 25, 23, 139, 161, 3389],
        'critical': {'ehr_app': ['tomcat', 'https'], 'patient_db': ['postgresql'], 'audit_db': ['mysql']},
        'dependencies': {
            'patient_portal': {'http': ['tomcat']},
            'ehr_app': {'tomcat': ['postgresql']},
        },
    },
    {
        'name': '06_financial_pci',
        'description': 'Compliance PCI-DSS',
        'hosts': [
            ('web_banking', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('api_gateway', [443, 3000, 22], ['https', 'nodejs', 'ssh'], False),
            ('core_banking', [8080, 69, 22], ['tomcat', 'tftp', 'ssh'], True),
            ('tx_processor', [8080, 443, 22], ['tomcat', 'https', 'ssh'], True),
            ('mq_cluster', [5672, 161, 22], ['rabbitmq', 'snmp', 'ssh'], False),
            ('accounts_db', [5432, 22, 23], ['postgresql', 'ssh'], True),
            ('tx_db', [5432, 22, 139], ['postgresql', 'ssh'], True),
            ('admin_console', [3389, 5900, 22], ['rdp', 'vnc', 'ssh'], False),
            ('security_siem', [9200, 5601, 22, 23], ['elasticsearch', 'kibana', 'ssh'], False),
        ],
        'forbidden': [80, 23, 139, 161, 69, 3389, 5900],
        'critical': {'core_banking': ['tomcat'], 'tx_processor': ['tomcat', 'https'], 
                     'accounts_db': ['postgresql'], 'tx_db': ['postgresql']},
        'dependencies': {
            'web_banking': {'http': ['nodejs']},
            'api_gateway': {'nodejs': ['tomcat', 'rabbitmq']},
            'tx_processor': {'tomcat': ['postgresql', 'rabbitmq']},
        },
    },
    {
        'name': '07_stress_test',
        'description': 'Test di scalabilità',
        'hosts': [
            ('web1', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('web2', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('web3', [80, 443, 22], ['http', 'https', 'ssh'], False),
            ('app1', [8080, 21, 161, 22], ['tomcat', 'ftp', 'snmp', 'ssh'], False),
            ('app2', [8080, 21, 161, 22], ['tomcat', 'ftp', 'snmp', 'ssh'], False),
            ('storage1', [445, 139, 22], ['smb', 'netbios', 'ssh'], False),
            ('storage2', [445, 139, 22], ['smb', 'netbios', 'ssh'], False),
            ('mail1', [25, 110, 22], ['smtp', 'pop3', 'ssh'], False),
            ('mail2', [25, 143, 22], ['smtp', 'imap', 'ssh'], False),
            ('admin1', [3389, 5900, 69, 22], ['rdp', 'vnc', 'tftp', 'ssh'], False),
            ('admin2', [3389, 5900, 22], ['rdp', 'vnc', 'ssh'], False),
            ('db1', [3306, 22, 23], ['mysql', 'ssh'], True),
            ('db2', [5432, 22, 23], ['postgresql', 'ssh'], True),
            ('db3', [27017, 22, 139], ['mongodb', 'ssh'], True),
        ],
        'forbidden': [80, 21, 23, 25, 139, 161, 69, 3389, 5900],
        'critical': {'db1': ['mysql'], 'db2': ['postgresql'], 'db3': ['mongodb']},
        'dependencies': {},
    },
    {
        'name': '08_impossible',
        'description': 'Scenario irrisolvibile (test robustezza)',
        'hosts': [
            ('critical_legacy', [3306, 22], ['mysql', 'ssh'], True),
        ],
        'forbidden': [3306],
        'critical': {'critical_legacy': ['mysql']},
        'dependencies': {},
    },
]

In [ ]:
def generate_scenarios():
    """Genera i file JSON degli scenari."""
    
    print("Generazione scenari...\n")
    
    for cfg in SCENARIOS:
        scenario = {
            'scenario_name': cfg['name'],
            'description': cfg['description'],
            'hosts': [{'id': h[0], 'open_ports': h[1], 'services': h[2], 'critical': h[3]} for h in cfg['hosts']],
            'policy': {'forbidden_ports': cfg['forbidden'], 'critical_services': cfg['critical']},
            'service_port_mapping': SERVICE_PORT_MAPPING,
            'service_dependencies': cfg['dependencies']
        }
        
        path = f"scenarios/{cfg['name']}.json"
        with open(path, 'w') as f:
            json.dump(scenario, f, indent=2)
        
        num_hosts = len(cfg['hosts'])
        num_forbidden = len(cfg['forbidden'])
        print(f"  {cfg['name']}: {num_hosts} host, {num_forbidden} porte vietate")
    
    print(f"\nGenerati {len(SCENARIOS)} scenari in 'scenarios/'")

generate_scenarios()

---
## 5. Esecuzione e Risultati <a id="5-esecuzione"></a>

### 5.1 Solver

Utilizzo di Fast-Downward con grounding e ottimizzazione dei costi.

In [ ]:
def solve(problem, timeout=300):
    """Risolve il problema usando Fast-Downward."""
    try:
        with Compiler(problem_kind=problem.kind, compilation_kind=CompilationKind.GROUNDING) as compiler:
            compiled = compiler.compile(problem)
            
            with OneshotPlanner(name='fast-downward') as planner:
                result = planner.solve(compiled.problem)
                
                if result.plan:
                    result.plan = result.plan.replace_action_instances(compiled.map_back_action_instance)
                
                return result, len(list(compiled.problem.actions))
    except Exception as e:
        print(f"  Errore: {str(e)[:100]}")
        return None, 0

### 5.2 Esecuzione di Tutti gli Scenari

In [ ]:
def get_action_type(action_name):
    """Classifica il tipo di azione."""
    name = action_name.lower()
    if 'chiudi_porta' in name:
        return 'chiudi'
    elif 'disattiva_servizio' in name:
        return 'stop'
    elif 'migra_servizio' in name:
        return 'migra'
    return 'altro'

def run_all_scenarios():
    """Esegue tutti gli scenari e raccoglie i risultati."""
    
    print("=" * 70)
    print("NETWORK HARDENING PLANNER")
    print("=" * 70)
    print(f"\nAzioni: chiudi_porta (costo {ACTION_COSTS['chiudi_porta']}), "
          f"disattiva_servizio (costo {ACTION_COSTS['disattiva_servizio']}), "
          f"migra_servizio (costo {ACTION_COSTS['migra_servizio']})\n")
    
    files = sorted([f for f in os.listdir('scenarios') if f.endswith('.json')])
    results = []
    
    for filename in files:
        with open(f'scenarios/{filename}') as f:
            scenario = json.load(f)
        
        name = scenario['scenario_name']
        print(f"\n{'─' * 70}")
        print(f"Scenario: {name}")
        print(f"  {scenario.get('description', '')}")
        
        problem, domain, objects = setup_problem(scenario)
        print(f"  Oggetti: {len(problem.all_objects)}, Azioni: {len(problem.actions)}, Goal: {len(problem.goals)}")
        
        start = time.time()
        result, ground_actions = solve(problem)
        elapsed = time.time() - start
        
        metrics = {
            'scenario': name,
            'status': 'ERROR',
            'time': round(elapsed, 3),
            'actions': 0,
            'cost': 0,
            'chiudi': 0,
            'stop': 0,
            'migra': 0
        }
        
        if result is None:
            print(f"  Risultato: ERRORE")
        elif result.status not in [PlanGenerationResultStatus.SOLVED_SATISFICING, 
                                    PlanGenerationResultStatus.SOLVED_OPTIMALLY]:
            metrics['status'] = 'UNSOLVABLE'
            print(f"  Risultato: NON RISOLVIBILE")
        else:
            metrics['status'] = 'SUCCESS'
            plan = [str(a) for a in result.plan.actions]
            metrics['actions'] = len(plan)
            
            for a in plan:
                t = get_action_type(a)
                if t == 'chiudi':
                    metrics['chiudi'] += 1
                    metrics['cost'] += ACTION_COSTS['chiudi_porta']
                elif t == 'stop':
                    metrics['stop'] += 1
                    metrics['cost'] += ACTION_COSTS['disattiva_servizio']
                elif t == 'migra':
                    metrics['migra'] += 1
                    metrics['cost'] += ACTION_COSTS['migra_servizio']
            
            print(f"  Risultato: RISOLTO in {elapsed:.3f}s")
            print(f"  Piano: {len(plan)} azioni, costo {metrics['cost']}")
            print(f"  Dettaglio: {metrics['migra']} migrazioni, {metrics['stop']} stop, {metrics['chiudi']} chiusure")
            
            # Salvataggio piano
            with open(f"results/plan_{name}.txt", 'w') as f:
                f.write(f"Piano: {name}\n{'=' * 50}\n\n")
                for i, a in enumerate(plan, 1):
                    f.write(f"{i:3d}. {a}\n")
        
        results.append(metrics)
    
    return pd.DataFrame(results)

df_results = run_all_scenarios()

---
## 6. Analisi dei Risultati <a id="6-analisi"></a>

### 6.1 Tabella Riepilogativa

In [ ]:
print("\n" + "=" * 70)
print("RIEPILOGO RISULTATI")
print("=" * 70 + "\n")
print(df_results.to_string(index=False))

# Salvataggio CSV
df_results.to_csv('results/summary.csv', index=False)
print("\nRisultati salvati in 'results/summary.csv'")

### 6.2 Statistiche

In [ ]:
print("\nSTATISTICHE")
print("-" * 40)

n_success = len(df_results[df_results['status'] == 'SUCCESS'])
n_fail = len(df_results[df_results['status'] != 'SUCCESS'])
df_ok = df_results[df_results['status'] == 'SUCCESS']

print(f"Scenari risolti:     {n_success}/{len(df_results)}")
print(f"Scenari irrisolvibili: {n_fail}")

if len(df_ok) > 0:
    print(f"\nTotale azioni:       {int(df_ok['actions'].sum())}")
    print(f"  - Migrazioni:      {int(df_ok['migra'].sum())}")
    print(f"  - Disattivazioni:  {int(df_ok['stop'].sum())}")
    print(f"  - Chiusure porte:  {int(df_ok['chiudi'].sum())}")
    print(f"\nCosto totale:        {int(df_ok['cost'].sum())}")
    print(f"Tempo medio:         {df_ok['time'].mean():.3f}s")

### 6.3 Grafici

In [ ]:
df_ok = df_results[df_results['status'] == 'SUCCESS']

if len(df_ok) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(12, 9))
    
    # Tempo di risoluzione
    colors = ['#2ecc71' if s == 'SUCCESS' else '#e74c3c' for s in df_results['status']]
    axes[0, 0].bar(range(len(df_results)), df_results['time'], color=colors)
    axes[0, 0].set_xticks(range(len(df_results)))
    axes[0, 0].set_xticklabels([s[:10] for s in df_results['scenario']], rotation=45, ha='right')
    axes[0, 0].set_ylabel('Tempo (s)')
    axes[0, 0].set_title('Tempo di Risoluzione')
    axes[0, 0].grid(axis='y', alpha=0.3)
    
    # Costo totale
    axes[0, 1].bar(range(len(df_ok)), df_ok['cost'], color='#3498db')
    axes[0, 1].set_xticks(range(len(df_ok)))
    axes[0, 1].set_xticklabels([s[:10] for s in df_ok['scenario']], rotation=45, ha='right')
    axes[0, 1].set_ylabel('Costo')
    axes[0, 1].set_title('Costo Totale')
    axes[0, 1].grid(axis='y', alpha=0.3)
    
    # Tipologia azioni
    x = range(len(df_ok))
    w = 0.25
    axes[1, 0].bar([i-w for i in x], df_ok['chiudi'], w, label='Chiusure', color='#e74c3c')
    axes[1, 0].bar([i for i in x], df_ok['stop'], w, label='Stop', color='#f39c12')
    axes[1, 0].bar([i+w for i in x], df_ok['migra'], w, label='Migrazioni', color='#2ecc71')
    axes[1, 0].set_xticks(x)
    axes[1, 0].set_xticklabels([s[:10] for s in df_ok['scenario']], rotation=45, ha='right')
    axes[1, 0].set_ylabel('Numero')
    axes[1, 0].set_title('Azioni per Tipo')
    axes[1, 0].legend()
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    # Success rate
    counts = df_results['status'].value_counts()
    colors_pie = ['#2ecc71' if s == 'SUCCESS' else '#e74c3c' for s in counts.index]
    axes[1, 1].pie(counts.values, labels=counts.index, autopct='%1.0f%%', colors=colors_pie)
    axes[1, 1].set_title('Tasso di Successo')
    
    plt.tight_layout()
    plt.savefig('results/analysis.png', dpi=150)
    plt.show()
    print("\nGrafico salvato in 'results/analysis.png'")
else:
    print("Nessuno scenario risolto.")

---
## Note Tecniche

### Scelte Progettuali

1. **Tre azioni invece di cinque**: La versione iniziale includeva anche `riattiva_servizio` e separava le migrazioni per servizi critici/non critici. Queste sono state rimosse perché:
   - `riattiva_servizio` non veniva mai usata (nessun goal richiede servizi attivi)
   - Le due migrazioni avevano lo stesso effetto, differivano solo nel costo

2. **Porte senza alternativa**: SMTP (25), NetBIOS (139), SNMP (161) e TFTP (69) non hanno alternative configurate. Questo forza il planner a usare `disattiva_servizio` per questi casi.

3. **Dipendenze tra servizi**: Se un servizio A dipende da B, A non può essere attivo se B è inattivo. Il planner deve fermare prima i servizi dipendenti.

4. **Servizi critici**: Non possono essere disattivati. Se un servizio critico è su una porta vietata senza alternativa, lo scenario è irrisolvibile.

### Limitazioni

- Il modello assume che le migrazioni siano atomiche (nessun downtime)
- Non sono gestite finestre di manutenzione temporali
- Le dipendenze sono limitate allo stesso host